1. Из ноутбуков по практике "Рекуррентные и одномерные сверточные нейронные сети" выберите лучшую сеть, либо создайте свою.
2. Запустите раздел "Подготовка"
3. Подготовьте датасет с параметрами `VOCAB_SIZE=20'000`, `WIN_SIZE=1000`, `WIN_HOP=100`, как в ноутбуке занятия, и обучите выбранную сеть. Параметры обучения можно взять из практического занятия. Для  всех обучаемых сетей в данной работе они должны быть одни и теже.
4. Поменяйте размер словаря tokenaizera (`VOCAB_SIZE`) на `5000`, `10000`, `40000`.  Пересоздайте датасеты, при этом оставьте `WIN_SIZE=1000`, `WIN_HOP=100`.
Обучите выбранную нейронку на этих датасетах.  Сделайте выводы об  изменении  точности распознавания авторов текстов. Результаты сведите в таблицу
5. Поменяйте длину отрезка текста и шаг окна разбиения текста на векторы  (`WIN_SIZE`, `WIN_HOP`) используя значения (`500`,`50`) и (`2000`,`200`). Пересоздайте датасеты, при этом оставьте `VOCAB_SIZE=20000`. Обучите выбранную нейронку на этих датасетах. Сделайте выводы об  изменении точности распознавания авторов текстов.

In [ ]:
# Работа с массивами данных
# Функции операционной системы
import os
# Регулярные выражения
import re
# Работа со временем
import time
import zipfile

# Загрузка датасетов из облака google
import gdown
# Отрисовка графиков
import matplotlib.pyplot as plt
import numpy as np
# Вывод объектов в ячейке colab
from IPython.display import display
# Матрица ошибок классификатора
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils
# Основные слои
from tensorflow.keras.layers import ( LSTM,BatchNormalization,
                                     Bidirectional,  Dense, Dropout,
                                     Embedding, 
                                     SpatialDropout1D)
# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
# Рисование схемы модели
from tensorflow.keras.utils import plot_model

%matplotlib inline


Функция для скачивания и распаковки датасета из облачного хранилища.
Параметры:
- url: URL архива с датасетом
- output_dir: директория для сохранения распакованных файлов
Действия:
1. Создает директорию, если она не существует
2. Скачивает архив по указанному URL
3. Распаковывает архив в указанную директорию
4. Удаляет скачанный архив

In [2]:
def download_and_extract(url, output_dir):
    """Скачивает и распаковывает архив с датасетом"""
    # Создаем папку для данных, если ее нет
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Путь для сохранения архива
    zip_path = os.path.join(output_dir, 'dataset.zip')
    
    # Скачиваем архив
    print("Скачивание архива...")
    gdown.download(url, zip_path, quiet=False)
    
    # Распаковываем архив
    print("Распаковка архива...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    
    # Удаляем архив
    os.remove(zip_path)
    print("Датасет готов к использованию")

Загрузка и распаковка датасета с текстами авторов.
Используется функция download_and_extract с указанием:
- URL архива в Yandex Cloud
- Локальной директории 'writers' для сохранения файлов

In [5]:
# Загрузка и распаковка архива
download_and_extract('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', 'writers')

Downloading...
From: https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip
To: c:\Users\HAIER\Desktop\Задания\Основы нейронных сетей\Занятие 5\nn_lab_5\ДЗ\writers\dataset.zip


Скачивание архива...


100%|██████████| 8.13M/8.13M [00:01<00:00, 4.41MB/s]


Распаковка архива...
Датасет готов к использованию


Настройка констант для загрузки и обработки данных:
- FILE_DIR: папка с текстовыми файлами
- SIG_TRAIN: подстрока в имени файла, указывающая на обучающую выборку
- SIG_TEST: подстрока в имени файла, указывающая на тестовую выборку

In [ ]:
# Настройка констант для загрузки данных
FILE_DIR  = 'writers'                     # Папка с текстовыми файлами
SIG_TRAIN = 'обучающая'                   # Признак обучающей выборки в имени файла
SIG_TEST  = 'тестовая'                    # Признак тестовой выборки в имени файла

Загрузка текстовых данных из файлов и их разделение на обучающую и тестовую выборки.
Алгоритм:
1. Инициализация пустых списков для классов и текстов
2. Получение списка файлов в директории
3. Обработка каждого файла:
   - Извлечение имени класса и типа выборки из имени файла
   - Добавление нового класса в CLASS_LIST при необходимости
   - Чтение содержимого файла
   - Добавление текста в соответствующую выборку (train/test)

In [26]:
# Подготовим пустые списки

CLASS_LIST = []  # Список классов
text_train = []  # Список для оучающей выборки
text_test = []   # Список для тестовой выборки

# Получим списка файлов в папке
file_list = os.listdir(FILE_DIR)

for file_name in file_list:
    # Выделяем имя класса и типа выборки из имени файла
    m = re.match('\((.+)\) (\S+)_', file_name)
    # Если выделение получилось, то файл обрабатываем
    if m:

        # Получим имя класса
        class_name = m[1]

        # Получим имя выборки
        subset_name = m[2].lower()

        # Проверим тип выборки
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая либо тестовая - файл обрабатываем
        if is_train or is_test:

            # Добавляем новый класс, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)

                # Инициализируем соответствующих классу строки текста
                text_train.append('')
                text_test.append('')

            # Найдем индекс класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')

            # Откроем файл на чтение
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:

                # Загрузим содержимого файла в строку
                text = f.read()
            # Определим выборку, куда будет добавлено содержимое
            subset = text_train if is_train else text_test

            # Добавим текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')

Добавление класса "Макс Фрай"
Добавление файла "(Макс Фрай) Тестовая_2 вместе.txt" в класс "Макс Фрай", тестовая выборка.
Добавление класса "Клиффорд_Саймак"
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вместе.txt" в класс "Клиффорд_Саймак", обучающая выборка.
Добавление класса "Рэй Брэдберри"
Добавление файла "(Рэй Брэдберри) Тестовая_8 вместе.txt" в класс "Рэй Брэдберри", тестовая выборка.
Добавление файла "(Клиффорд_Саймак) Тестовая_2 вместе.txt" в класс "Клиффорд_Саймак", тестовая выборка.
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление класса "Стругацкие"
Добавление файла "(Стругацкие) Обучающая_5 вместе.txt" в класс "Стругацкие", обучающая выборка.
Добавление класса "О. Генри"
Добавление файла "(О. Генри) Тестовая_20 вместе.txt" в класс "О. Генри", тестовая выборка.
Добавление класса "Булгаков"
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление файла "(Рэй Брэдберри) Обуч

Определение количества классов на основе загруженных данных.
CLASS_COUNT будет использоваться при построении модели.

In [27]:
# Определим количество классов
CLASS_COUNT = len(CLASS_LIST)

Вывод списка загруженных классов текстов для проверки корректности загрузки.

In [28]:
# Выведем прочитанные классы текстов
print(CLASS_LIST)

['Макс Фрай', 'Клиффорд_Саймак', 'Рэй Брэдберри', 'Стругацкие', 'О. Генри', 'Булгаков']


Проверка количества текстов в обучающей выборке.
Должно соответствовать количеству классов.

In [29]:
# Посчитаем количество текстов в обучающей выборке
print(len(text_train))

6


Проверка загруженных данных: вывод начальных фрагментов текстов 
из каждого класса для обучающей и тестовой выборок.

In [30]:
# Проверим загрузки: выведем начальные отрывки из каждого класса

for cls in range(CLASS_COUNT):                   # Запустим цикл по числу классов
    print(f'Класс: {CLASS_LIST[cls]}')           # Выведем имя класса
    print(f'  train: {text_train[cls][:200]}')   # Выведем фрагмент обучающей выборки
    print(f'  test : {text_test[cls][:200]}')    # Выведем фрагмент тестовой выборки
    print()

Класс: Макс Фрай
  train:  ﻿Власть несбывшегося   – С тех пор как меня угораздило побывать в этой грешной Черхавле, мне ежедневно снится какая-то дичь! – сердито сказал я Джуффину. – Сглазили они меня, что ли? А собственно, по
  test :  ﻿Слишком много кошмаров    Когда балансируешь над пропастью на узкой, скользкой от крови доске, ответ на закономерный вопрос: «Как меня сюда занесло?» – вряд ли принесёт практическую пользу. Зато пои

Класс: Клиффорд_Саймак
  train:  ﻿Всё живое...     Когда я выехал из нашего городишка и повернул на шоссе, позади оказался грузовик. Этакая тяжелая громадина с прицепом, и неслась она во весь дух. Шоссе здесь срезает угол городка, и
  test :  ﻿Зачарованное паломничество    1  Гоблин со стропил следил за прячущимся монахом, который шпионил за ученым. Гоблин ненавидел монаха и имел для этого все основания. Монах никого не ненавидел и не люб

Класс: Рэй Брэдберри
  train:  ﻿451° по Фаренгейту   ДОНУ КОНГДОНУ С БЛАГОДАРНОСТЬЮ   Если тебе дадут линованную бумаг

Контекстный менеджер для измерения времени выполнения операций.

In [31]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with

class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

Определение параметров для экспериментов:
- VOCAB_SIZES: размеры словаря для токенизатора
- WIN_SIZES: размеры окон для сегментации текста
- WIN_HOPS: шаги окон для сегментации текста
- EPOCHS: количество эпох обучения
- BATCH_SIZE: размер батча

In [32]:
VOCAB_SIZES = [5000, 10000, 20000, 40000]  # изменение размера словаря токенизатора
WIN_SIZES = [1000, 500, 2000]   # Размеры окна сегментации текста
WIN_HOPS  = [100, 50, 200]  # Шаг окна сегментации текста

EPOCHS = 5
BATCH_SIZE = 128

Функция для подготовки датасета с заданными параметрами.
Параметры:
- vocab_size: размер словаря токенизатора
- win_size: размер окна сегментации текста
- win_hop: шаг окна сегментации текста

Возвращает:
- X_train, X_test: последовательности для обучающей и тестовой выборок
- y_train, y_test: метки классов в one-hot формате
- tokenizer: обученный токенизатор

Алгоритм:
1. Создание и обучение токенизатора
2. Разбивка текстов на последовательности фиксированной длины
3. Преобразование меток классов в one-hot формат
4. Добавление паддинга к последовательностям

In [ ]:
# Функция подготовки датасета
# Подготовка датасета с разными VOCAB_SIZE, WIN_SIZE, WIN_HOP.
def prepare_dataset(vocab_size, win_size, win_hop):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
    tokenizer.fit_on_texts(text_train + text_test)

    # Разбивка текста на отрезки фиксированной длины с заданным шагом.
    # Реализация параметров WIN_SIZE, WIN_HOP.
    def split_texts(texts):
        sequences = []
        labels = []
        for idx, text in enumerate(texts):
            seq = tokenizer.texts_to_sequences([text])[0]
            for i in range(0, len(seq) - win_size, win_hop):
                chunk = seq[i:i+win_size]
                sequences.append(chunk)
                labels.append(idx)
        return sequences, labels

    # Подготовка входных и выходных данных
    # Паддинг последовательностей и one-hot-кодировка меток.
    X_train, y_train = split_texts(text_train)
    X_test, y_test = split_texts(text_test)

    X_train = pad_sequences(X_train, maxlen=win_size)
    X_test = pad_sequences(X_test, maxlen=win_size)

    y_train = utils.to_categorical(y_train, CLASS_COUNT)
    y_test = utils.to_categorical(y_test, CLASS_COUNT)

    return np.array(X_train), np.array(X_test), y_train, y_test, tokenizer


Функция для создания модели нейронной сети.
Архитектура:
1. Embedding слой для преобразования токенов в векторы
2. SpatialDropout1D для регуляризации
3. BatchNormalization для нормализации активаций
4. Два Bidirectional LSTM слоя для обработки последовательностей
5. Dropout для регуляризации
6. Dense слой с softmax активацией для классификации

Параметры:
- VOCAB_SIZE: размер словаря
- WIN_SIZE: длина входных последовательностей

Возвращает скомпилированную модель с оптимизатором Adam и метрикой accuracy.

In [34]:
def build_model(VOCAB_SIZE, WIN_SIZE):
    model_LSTM_7 = Sequential()
    model_LSTM_7.add(Embedding(VOCAB_SIZE, 50, input_length=WIN_SIZE))
    model_LSTM_7.add(SpatialDropout1D(0.4))
    model_LSTM_7.add(BatchNormalization())
    # Два двунаправленных рекуррентных слоя LSTM
    model_LSTM_7.add(Bidirectional(LSTM(8, return_sequences=True)))
    model_LSTM_7.add(Bidirectional(LSTM(8)))
    model_LSTM_7.add(Dropout(0.3))
    model_LSTM_7.add(Dense(CLASS_COUNT, activation='softmax'))
    model_LSTM_7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_LSTM_7

Эксперимент с разными размерами словаря (VOCAB_SIZE).
Алгоритм:
1. Для каждого размера словаря из VOCAB_SIZES:
   - Подготовка датасета с фиксированными WIN_SIZE=1000 и WIN_HOP=100
   - Построение и обучение модели
   - Оценка точности на тестовой выборке
   - Сохранение результатов
2. Вывод таблицы с результатами

Используется контекстный менеджер timex для измерения времени выполнения.

In [35]:
results_vocab = []

# Задание: VOCAB_SIZE изменение
for vocab_size in VOCAB_SIZES:
    print(f'\n\n=== VOCAB_SIZE = {vocab_size} ===')
    with timex():
        X_train, X_test, y_train, y_test, _ = prepare_dataset(vocab_size, 1000, 100)
        model = build_model(vocab_size, 1000)
        history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                            validation_data=(X_test, y_test), verbose=0)
        acc = model.evaluate(X_test, y_test, verbose=0)[1]
        results_vocab.append((vocab_size, acc))
        print(f'Accuracy: {acc:.4f}')

# Таблица результатов по VOCAB_SIZE
print("\nТочность для разных размеров словаря:")
for vocab_size, acc in results_vocab:
    print(f'VOCAB_SIZE={vocab_size:<6} --> Accuracy={acc:.4f}')



=== VOCAB_SIZE = 5000 ===
Accuracy: 0.4462
Время обработки: 112.02 с


=== VOCAB_SIZE = 10000 ===
Accuracy: 0.5296
Время обработки: 111.22 с


=== VOCAB_SIZE = 20000 ===
Accuracy: 0.3670
Время обработки: 112.54 с


=== VOCAB_SIZE = 40000 ===
Accuracy: 0.4945
Время обработки: 112.87 с

Точность для разных размеров словаря:
VOCAB_SIZE=5000   --> Accuracy=0.4462
VOCAB_SIZE=10000  --> Accuracy=0.5296
VOCAB_SIZE=20000  --> Accuracy=0.3670
VOCAB_SIZE=40000  --> Accuracy=0.4945


Эксперимент с разными параметрами окон (WIN_SIZE, WIN_HOP).
Алгоритм:
1. Для каждой пары (WIN_SIZE, WIN_HOP):
   - Подготовка датасета с фиксированным VOCAB_SIZE=20000
   - Построение и обучение модели
   - Оценка точности на тестовой выборке
   - Сохранение результатов
2. Вывод таблицы с результатами

Используется контекстный менеджер timex для измерения времени выполнения.

In [36]:
results_win = []

for win_size, win_hop in [(500,50), (2000,200)]:
    print(f'\n\n=== WIN_SIZE = {win_size}, WIN_HOP = {win_hop} ===')
    with timex():
        X_train, X_test, y_train, y_test, _ = prepare_dataset(20000, win_size, win_hop)
        model = build_model(20000, win_size)
        history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                            validation_data=(X_test, y_test), verbose=0)
        acc = model.evaluate(X_test, y_test, verbose=0)[1]
        results_win.append((win_size, win_hop, acc))
        print(f'Accuracy: {acc:.4f}')

# Таблица результатов по WIN_SIZE/WIN_HOP
print("\nТочность для разных параметров окна:")
for win_size, win_hop, acc in results_win:
    print(f'WIN_SIZE={win_size:<5}, WIN_HOP={win_hop:<4} --> Accuracy={acc:.4f}')



=== WIN_SIZE = 500, WIN_HOP = 50 ===
Accuracy: 0.5372
Время обработки: 120.82 с


=== WIN_SIZE = 2000, WIN_HOP = 200 ===
Accuracy: 0.3548
Время обработки: 107.65 с

Точность для разных параметров окна:
WIN_SIZE=500  , WIN_HOP=50   --> Accuracy=0.5372
WIN_SIZE=2000 , WIN_HOP=200  --> Accuracy=0.3548
